In [1]:
import qutip as qt
import numpy as np
print(qt.__version__)



5.0.4


## First we define a random 3 qubit state and its density matrix

In [ ]:
def create_random_state(n):
    basis_states = [qt.basis([2]*n, [int(i) for i in format(j, f'0{n}b')]) for j in range(2**n)]
    
    random_probs = np.random.rand(2**n)
    random_probs /= np.sum(random_probs)  # Normalize to sum to 1
    
    coefficients = np.sqrt(random_probs)
   
    random_state = sum(coeff * basis for coeff, basis in zip(coefficients, basis_states))
    return random_state

def compute_density_matrix(state):
    return state * state.dag()

num_qubits = 3  

random_state = create_random_state(num_qubits)

p_random = compute_density_matrix(random_state)

S_random = qt.entropy_vn(p_random)

print("Random state:")
print(random_state)
np.set_printoptions(linewidth=200, precision=3, suppress=True)
print("Density Matrix in Matrix Form:")
print(np.real(p_random.full()))
print("\nEntropy of the total system S(A,B,C) =", S_random)



Random state:
Quantum object: dims=[[2, 2, 2], [1, 1, 1]], shape=(8, 1), type='ket', dtype=Dense
Qobj data =
[[0.4665109 ]
 [0.08250118]
 [0.17936138]
 [0.42200354]
 [0.23716733]
 [0.24956882]
 [0.42094372]
 [0.51920813]]
Density Matrix in Matrix Form:
[[0.218 0.038 0.084 0.197 0.111 0.116 0.196 0.242]
 [0.038 0.007 0.015 0.035 0.02  0.021 0.035 0.043]
 [0.084 0.015 0.032 0.076 0.043 0.045 0.076 0.093]
 [0.197 0.035 0.076 0.178 0.1   0.105 0.178 0.219]
 [0.111 0.02  0.043 0.1   0.056 0.059 0.1   0.123]
 [0.116 0.021 0.045 0.105 0.059 0.062 0.105 0.13 ]
 [0.196 0.035 0.076 0.178 0.1   0.105 0.177 0.219]
 [0.242 0.043 0.093 0.219 0.123 0.13  0.219 0.27 ]]

Entropy of the total system S(A,B,C) = 5.19230331189701e-15


In [ ]:
def verify_normalization(state):
    """Computes the sum of the squares of the coefficients of a quantum state."""
    coefficients = state.full().flatten()  
    norm_sum = np.sum(np.abs(coefficients) ** 2)  
    return norm_sum

norm_check = verify_normalization(random_state)
print(f"Sum of squared coefficients: {norm_check:.6f}")  


Sum of squared coefficients: 1.000000


## Then we compute partial traces to get rdms and compute entropies

In [4]:

p_A = p_random.ptrace([0])  
p_B = p_random.ptrace([1])  
p_C = p_random.ptrace([2])  


S_A = qt.entropy_vn(p_A)
S_B = qt.entropy_vn(p_B)
S_C = qt.entropy_vn(p_C)

I_ABC = S_A + S_B + S_C - S_random

np.set_printoptions(linewidth=200, precision=3, suppress=True)

print("Reduced Density Matrix ρ_A:\n", np.real(p_A.full()), "\n",f"Von Neumann Entropy S(ρ_A): {S_A:.6f} \n")
print("Reduced Density Matrix ρ_B:\n", np.real(p_B.full()), "\n",f"Von Neumann Entropy S(ρ_B): {S_B:.6f} \n")
print("Reduced Density Matrix ρ_C:\n", np.real(p_C.full()), "\n",f"Von Neumann Entropy S(ρ_C): {S_C:.6f} \n")
print("\n \n Total Information I(A:B:C) = S(A) + S(B) + S(C) - S(A,B,C) = ",I_ABC)

Reduced Density Matrix ρ_A:
 [[0.435 0.426]
 [0.426 0.565]] 
 Von Neumann Entropy S(ρ_A): 0.251519 

Reduced Density Matrix ρ_B:
 [[0.343 0.348]
 [0.348 0.657]] 
 Von Neumann Entropy S(ρ_B): 0.363527 

Reduced Density Matrix ρ_C:
 [[0.483 0.392]
 [0.392 0.517]] 
 Von Neumann Entropy S(ρ_C): 0.341714 


 
 Total Information I(A:B:C) = S(A) + S(B) + S(C) - S(A,B,C) =  0.9567595060825086


## Now we work with some operators correlations, first Z:

In [5]:
Z = qt.sigmaz()

Z_full = qt.tensor(Z, Z, Z)
avg_Z_full = (p_random * Z_full).tr()

avg_Z_A = (p_A * Z).tr()
avg_Z_B = (p_B * Z).tr()
avg_Z_C = (p_C * Z).tr()

Z_norm = Z.norm()

print(f"Tr(p_W * (σ_z ⊗ σ_z ⊗ σ_z)): {avg_Z_full:.6f}")
print(f"Tr_A(p_A * σ_z): {avg_Z_A:.6f}")
print(f"Tr_B(p_B * σ_z): {avg_Z_B:.6f}")
print(f"Tr_C(p_C * σ_z): {avg_Z_C:.6f}")
print(f"Operator norm of σ_z: {Z_norm:.6f}")


Tr(p_W * (σ_z ⊗ σ_z ⊗ σ_z)): 0.270395+0.000000j
Tr_A(p_A * σ_z): -0.130607+0.000000j
Tr_B(p_B * σ_z): -0.314056+0.000000j
Tr_C(p_C * σ_z): -0.033510+0.000000j
Operator norm of σ_z: 2.000000


In [7]:
C_ABC_z = avg_Z_full - avg_Z_A*avg_Z_B*avg_Z_C
f_ABC_z = (C_ABC_z**2)/(2*3*Z_norm**2)

print("Concluding for Z we have that I(A,B,C) = ", I_ABC, "While the lower bound is f_ABC = ", np.real(f_ABC_z))

Concluding for Z we have that I(A,B,C) =  0.9567595060825086 While the lower bound is f_ABC =  0.0030774504215273


## Then X

In [8]:
x = qt.sigmax()

x_full = qt.tensor(x, x, x)
avg_x_full = (p_random * x_full).tr()

avg_x_A = (p_A * x).tr()
avg_x_B = (p_B * x).tr()
avg_x_C = (p_C * x).tr()

x_norm = x.norm()

print(f"Tr(p_W * (σ_x ⊗ σ_x ⊗ σ_x)): {avg_x_full:.6f}")
print(f"Tr_A(p_A * σ_x): {avg_x_A:.6f}")
print(f"Tr_B(p_B * σ_x): {avg_x_B:.6f}")
print(f"Tr_C(p_C * σ_x): {avg_x_C:.6f}")
print(f"Operator norm of σ_x: {x_norm:.6f}")

Tr(p_W * (σ_x ⊗ σ_x ⊗ σ_x)): 0.843586+0.000000j
Tr_A(p_A * σ_x): 0.851679+0.000000j
Tr_B(p_B * σ_x): 0.695804+0.000000j
Tr_C(p_C * σ_x): 0.783852+0.000000j
Operator norm of σ_x: 2.000000


In [9]:
C_ABC_x = avg_x_full - avg_x_A*avg_x_B*avg_x_C
f_ABC_x = (C_ABC_x**2)/(2*3*x_norm**2)

print("Concluding for X we have that I(A,B,C) = ", I_ABC, "While the lower bound is f_ABC = ", np.real(f_ABC_x))

Concluding for X we have that I(A,B,C) =  0.9567595060825086 While the lower bound is f_ABC =  0.005987384231554014


## And for Y:

In [11]:
y = qt.sigmay()

y_full = qt.tensor(y, y, y)
avg_y_full = (p_random * y_full).tr()

avg_y_A = (p_A * y).tr()
avg_y_B = (p_B * y).tr()
avg_y_C = (p_C * y).tr()

y_norm = y.norm()

print(f"Tr(p_W * (σ_y ⊗ σ_y ⊗ σ_y)): {avg_y_full:.6f}")
print(f"Tr_A(p_A * σ_y): {avg_y_A:.6f}")
print(f"Tr_B(p_B * σ_y): {avg_y_B:.6f}")
print(f"Tr_C(p_C * σ_y): {avg_y_C:.6f}")
print(f"Operator norm of σ_y: {y_norm:.6f}")

Tr(p_W * (σ_y ⊗ σ_y ⊗ σ_y)): 0.000000+0.000000j
Tr_A(p_A * σ_y): 0.000000+0.000000j
Tr_B(p_B * σ_y): 0.000000+0.000000j
Tr_C(p_C * σ_y): 0.000000+0.000000j
Operator norm of σ_y: 2.000000


In [13]:
C_ABC_y = avg_y_full - avg_y_A*avg_y_B*avg_y_C
f_ABC_y = (C_ABC_y**2)/(2*3*y_norm**2)

print("Concluding for Y we have that I(A,B,C) = ", I_ABC, "While the lower bound is f_ABC = ", np.real(f_ABC_y))

Concluding for Y we have that I(A,B,C) =  0.9567595060825086 While the lower bound is f_ABC =  -3.209883240645393e-35


## What if I try to mix them?

In [16]:
Mix_operator = qt.tensor(x, Z, Z)
full_mix_avg = (p_random * Mix_operator).tr()
C_ABC_MIX = full_mix_avg - avg_x_A*avg_Z_B*avg_Z_C
f_ABC_MIX = (C_ABC_MIX**2)/(2*x_norm**2*Z_norm**2*Z_norm**2)
print("Concluding for a mix of the 3 operators we have that I(A,B,C) = ", I_ABC, "While the lower bound is f_ABC = ", np.real(f_ABC_MIX))

Concluding for a mix of the 3 operators we have that I(A,B,C) =  0.9567595060825086 While the lower bound is f_ABC =  0.0016413079693077667
